<a href="https://colab.research.google.com/github/hida0/ESAA-2022-2/blob/HW/Week07_HW2_Text_Analysis_20newsgroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 개념 필사
(파이썬 머신러닝 완벽 가이드 ch8. 4 pg. 487~496)

## 8. 텍스트 분류 실습 - 20 뉴스그룹 분류
- 텍스트 분류는 특성 문서의 분류를 학습 데이터를 통해 학습하여 모델을 생성한 뒤, 이 학습 모델을 이용해 다른 문서의 분류를 예측하는 것이다.
- 사이킷런이 내부에 가지고 있는 예제 데이터인 20 뉴스그룹 데이터 세트를 이용하여 텍스트 분류를 적용한다.  사이킷런은 **`fetch_20newsgroups( )`** API를 이용하여 뉴스그룹의 분류를 수행해 볼 수 있는 예제 데이터를 제공한다. 
- **텍스트를 기반으로 분류**를 수행할 때는 먼저 **텍스트를 정규화**한 뒤 **피처 벡터화를 적용**한다. 그리고 그 이후에 적합한 **머신러닝 알고리즘을 적용하여 분류를 학습/예측/평가**한다.
- 텍스트를 피처 벡터화로 변환하면 일반적으로 **희소 행렬** 형태가 된다. 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘으로는 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등이 있다. 이중 **로지스틱 회귀**를 이용하여 분류를 수행한다.
---
- 카운트 기반과 TF-IDF 기반의 벡터화를 차례로 적용하여 예측 성능을 비교하고, 피처 벡터화를 위한 파라미터와 GridSearchCV 기반의 하이퍼 파라미터를 튜닝한다.
- 또한 사이킷런의 Pipeline 객체를 통해 피처 벡터화 파라미터와 GridSearchCV 기반의 하이퍼 파라미터 튜닝을 한꺼번에 수행하는 방법을 소개한다.


### 1. 텍스트 정규화
- **`fetch_20newsgroups( )`**는 사이킷런의 다른 데이터 세트 예제와 같이 파이썬 딕셔너리와 유사한 Bunch 객체를 반환한다.
- **`keys( )`**를 이용하여 key 값을 살펴보면 `load_xxx( )` API와 유사한 key 값을 가지고 있음을 알 수 있다. 
- `fetch_20newsgroups( )`는 인터넷에서 로컬 컴퓨터로 데이터를 먼저 내려받은 후에 메모리로 데이터를 로딩한다. 'filenames'라는 key는 이때 데이터를 저장하는 디렉터리와 파일명을 지칭한다.
- target 클래스의 값은 0부터 19까지 20개로 구성되어 있으며, 클래스 이름들의 앞에서부터 0, 1,.. 값을 가진다.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all', random_state=156)

In [ ]:
news_data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
import pandas as pd
print("target 클래스의 값과 분포도 \n,", pd.Series(news_data.target).value_counts().sort_index())

target 클래스의 값과 분포도 
, 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64


In [ ]:
print("target 클래스의 이름들 \n,", news_data.target_names)

target 클래스의 이름들 
, ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### 1. 텍스트 정규화
- 개별 데이터가 텍스트로 어떻게 구성되어 있는지 확인하기 위해 데이터를 1개만 추출한다.
- 텍스트 데이터를 확인해보면 뉴스그룹 기사의 내용뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등 다양한 정보를 가지고 있다.
- 이 중에서 내용을 제외하고 제목 등 다른 정보는 제거하는데, 제목과 소속, 이메일 주소 등의 헤더(headers)와 푸터(footers) 정보들은 뉴스그룹 분류의 target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많기 때문이다. 그렇기 때문에 이 피처들을 포함하게 되면 왠만한 ML 알고리즘을 적용해도 상당히 높은 예측 성능을 나타낸다.
---
- 따라서 **`remove`** 파라미터를 이용하면 뉴스그룹 기사의 헤더와 푸터 등을 제거하고, 순수한 텍스트만으로 구성된 기사 내용으로 어떤 뉴스그룹에 속하는지 분류한다.
- 또한 `fetch_20newsgroups( )`는 **`subset`** 파라미터를 이용하여 학습 데이터 세트와 테스트 데이터 세트를 분리해 내려 받을 수 있다.

In [ ]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [ ]:
from sklearn.datasets import fetch_20newsgroups
# subset='train'으로 학습용 데이터만, remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target

# subset='test'으로 학습용 데이터만, remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=156)
X_test = test_news.data
y_test = test_news.target

print("학습 데이터 크기 {0}, 테스트 데이터 크기 {1}".format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


### 2. 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
- 학습 데이터는 11314개의 뉴스그룹 문서가 리스트 형태로 주어지고, 테스트 데이터는 7532개의 문서가 리스트 형태로 주어졌다.
- **`CountVectorizer`**를 이용하여 **학습 데이터의 텍스트를 피처 벡터화**한다. 
- 테스트 데이터의 텍스트를 **`CountVectorizer`**를 이용하여 피처 벡터화할 때는 **반드시 학습 데이터를 이용하여 `fit( )`이 수행된 객체를 이용하여 테스트 데이터를 변환**(transform)해야한다. 그래야 학습 시에 설정된 `CountVectorizer`의 피처 개수와 테스트 데이터를 `CountVectorizer`로 변환할 피처의 개수가 같아진다. 
- 즉 **테스트 데이터의 피처 벡터화**는 **학습 데이터에 사용된** **`CountVectorizer`** **객체 변수**인 **cnt_vect.transform( )을 이용해 변환**한다.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Count Vectorization으로 학습 데이터 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit( )된 CountVectorizer를 이용하여 테스트 데이터 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print("학습 데이터 텍스트의 CountVectorizer Shape:", X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


### 2. 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
- 학습 데이터를 **`CountVectorizer`**로 피처를 추출한 결과 11314개의 문서에서 피처, 즉 단어가 101631개로 만들어졌다.
- 이렇게 피처 벡터화된 데이터에 로지스틱 회귀를 적용하여 뉴스그룹에 대한 분류를 예측한다.
- Count 기반으로 피처 벡터화가 적용된 데이터 세트에 대한 로지스틱 회귀의 예측 정확도는 약 0.597이다.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용하여 학습/예측/평가를 수행
lr_clf = LogisticRegression(max_iter=700)
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print("CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.597


### 2. 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
- 이번에는 TF-IDF 기반으로 피처 벡터화를 적용하여 예측 모델을 학습/예측/평가한다.
- TF-IDF 기반으로 피처 벡터화가 적용된 데이터 세트에 대한 로지스틱 회귀의 예측 정확도는 약 0.674이다.
- TF-IDF 기반의 피처 벡터화가 단순 Count 기반의 피처 벡터화보다 더 높은 예측 정확도를 제공한다.
- 일반적으로 문서 내에 텍스트가 많고, 많은 문서를 가지는 텍스트 분석에서 **Count 벡터화보다는 TF-IDf 벡터화가 더 좋은 예측 결과**를 도출한다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF 벡터화를 적용하여 학습 데이터 세트와 테스트 데이터 세트를 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용하여 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print("TF-IDFVectorized Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))

TF-IDFVectorized Logistic Regression의 예측 정확도는 0.674


### 2. 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
- 텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법은 최적의 ML 알고리즘을 선택하는 것과 최상의 피처 전처리를 수행하는 것이다.
- 텍스트 정규화나 Count/TF-IDF 기반의 피처 벡터화를 어떻게 효과적으로 적용했는지가 텍스트 기반의 머신러닝 성능에 큰 영향을 미칠 수 있다.
- 앞의 TF-IDF 벡터화는 기본 파라미터만 적용했지만, 더 다양한 파라미터를 적용할 수 있다.
- **`TfidfVectorizer`** 클래스의 **`stop_words`**를 기존 'None'에서 'english'로 변경하고, **`ngram_range`**는 기존 (1, 1)에서 (1, 2)로, **`max_df`**를 300으로 변경한 뒤 다시 예측 성능을 측정한다.
- 하이퍼 파라미터를 튜닝한 로지스틱 회귀의 예측 정확도는 약 0.692로, 튜닝하지 않은 모델보다 더 높은 정확도를 나타냈다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# stop_words 필터링을 추가하고 ngram을 (1, 1)에서 (1, 2)로 변경하여 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용하여 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print("TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.692


### 2. 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
- 이번에는 **`GridSearchCV`**를 이용하여 로지스틱 회귀의 하이퍼 파라미터 최적화를 수행한다.
- 로지스틱 회귀의 **`C`** 파라미터만 변경하면서 최적의  C값을 찾은 뒤, 이 C값으로 학습된 모델에서 테스트 데이터로 예측하여 성능을 평가한다.
- 로지스틱 회귀의 **`C`**가 10일때, GridSearchCV의 교차 검증 테스트 세트에서 가장 좋은 예측 성능을 나타냈다.
- 이를 테스트 데이터 세트에 적용하여 평가한 정확도는 0.701로, 이전보다 약간 향상되었다.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# 최적 C값을 도출하는 튜닝 수행, CV는 3 폴드 세트로 설정
lr_clf = LogisticRegression(max_iter=600)
params = {'C' : [5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print("Logistic Regression best C parameter :", grid_cv_lr.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Logistic Regression best C parameter : {'C': 10}


In [ ]:
# 최적 C값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print("TF-IDFVectorized Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))

TF-IDFVectorized Logistic Regression의 예측 정확도는 0.701


### 3. 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합
- 사이킷런의 **`Pipeline`** 클래스를 이용하면 **피처 벡터화와 ML 알고리즘 학습/예측**을 위한 코드를 한번에 작성할 수 있다.
- 일반적으로 머신러닝에서 Pipeline이란 데이터의 가공, 변환 등의 전처리와 알고리즘 적용을 한꺼번에 스트림 기반으로 처리한다는 의미이다. 
- 이렇게 Pipeline을 이용하면 **데이터의 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리**할 수 있어 더 직관적인 ML 모델 코드를 생성할 수 있다.
- 또한 대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 스트림 기반에서 바로 머신러닝 알고리즘의 데이터로 입력할 수 있기 때문에 수행 시간을 절약할 수 있다.
- 사이킷런 **`Pipeline`**은 **텍스트 기반의 피처 벡터화** 뿐만 아니라 **모든 데이터 전처리 작업과 Estimator를 결합**할 수 있다.
---
- 다음 코드는 위의 텍스트 분류 예제 코드를  **`Pipeline`**을 이용하여 다시 작성한 것이다.
-  **`Pipeline`** 객체는 다음과 같이 선언한다.
 - `pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])`
 - 이것은 TfidfVectorizer 객체를 tfidf_vect라는 객체 변수 명으로, LogisticRegression 객체를 lr_clf라는 객체 변수 명으로 생성한 뒤 이 2개의 객체를 파이프라인으로 연결하는 Pipeline 객체 pipeline을 생성한다는 의미이다.
- 또한 기존 **`TfidfVectorizer`**의 학습 데이터와 테스트 데이터에 대한 **`fit( )`과 `transform( )`** 수행을 통한 **피처 벡터화**와 **`LogisticRegression`**의 **`fit( )`과 `transform( )`** 수행을 통한 **머신러닝 모델의 학습과 예측**이 **`Pipeline`**의 **`fit( )`과 `predict( )`** 으로 통일되어 수행된다.
- 이렇게 Pipeline 방식을 적용하여 머신러닝 코드를 더 직관적이고 쉽게 작성할 수 있다.
- 파이프라인을 이용한 로지스틱 회귀의 예측 정확도는 약 0.701이다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10, max_iter=700))
])

# 별도의 TfidfVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), transform()이 필요없음
# pipeline의 fit(), predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print("Pipeline을 이용한 Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))

Pipeline을 이용한 Logistic Regression의 예측 정확도는 0.701


### 3. 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합
- 사이킷런은 **`GridSearchCV`** 클래스의 **생성 파라미터로 Pipeline을 입력**하여, Pipeline 기반에서도 하이퍼 파라미터 튜닝을 GridSearchCV 방식으로 진행할 수 있게 지원한다. 
- 이렇게 하면 **피처 벡터화를 위한 파라미터와 ML 알고리즘의 하이퍼 파라미터**를 모두 한번에 **`GridSearchCV`**를 **이용하여 최적화**할 수 있다.
---
- 다음 예제는 **`GridSearchCV`**에 **Pipeline**을 입력하면서 **TfidfVectorizer의 파라미터와 LogisticRegression의 하이퍼 파라미터를 함께 최적화**한다. 
-  **`GridSearchCV`**에 **Estimator**가 아닌 **Pipeline**을 입력할 경우에는 **param_grid`**의 입력 값 설정이 기존과 약간 다르다. 
- **딕셔너리 형태의 key와 value** 값을 가지며, **value를 리스트 형태로 입력**하는 것은 동일하다. 그러나 key 값을 살펴보면 'tfidf_vect__ngram_range'와 같이 **하이퍼 파라미터명이 객체 변수명과 결합되어 제공**된다.
- Pipeline을 **`GridSearchCV`**에 인자로 입력하면, **GridSearchCV**는 **Pipeline을 구성하는 피처 벡터화 객체의 파라미터와 Estimator 객체의 하이퍼 파라미터를 각각 구별**할 수 있어야 하므로, 이때 **개별 객체명과 하이퍼 파라미터명을 결합하여 key 값으로 할당**하는 것이다.
- **`Pipeline + GridSearchCV`**를 적용할 때 유의할 점은 모두의 파라미터를 최적화하려면 너무 많은 튜닝 시간이 소모된다는 점이다. 피처 벡터화에 사용되는 파라미터와 GridSearchCV 하이퍼 파라미터를 합치면 최적화를 위한 너무 많은 경우의 수가 발생하기 쉽다.
- 다음 코드의 경우 8개의 파라미터 경우의 수 X 3개의 CV로 총 24번의 학습과 검증을 수행하여 오랜 시간이 소요되었다.
- **`Pipeline + GridSearchCV`**을 이용한 로지스틱 회귀의 예측 정확도는 약 0.701이다.

In [ ]:
# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression(max_iter=600))
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(__) 2개를 붙여 GridSearchCv에 사용될 하이퍼 파라미터 이름과 값을 결정
params = {'tfidf_vect__ngram_range': [(1, 1), (1, 2)],
                    'tfidf_vect__max_df': [300, 700],
                    'lr_clf__C': [5, 10]}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체를 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)
pred = grid_cv_pipe.predict(X_test)